In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch

# 下载数据集的路径
path = "./DataSet"
# 使用的设备
device = torch.device("cuda")
# 数据预处理,转换为张量并归一化
transform = transforms.Compose(
    [
        transforms.Resize((224,   224)),  # 将图像大小调整为 224x224
        transforms.ToTensor(),  # 将图像转换为张量
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)  # 归一化处理

train_set = CIFAR10(root=path, train=True, download=True, transform=transform)
test_set = CIFAR10(root=path, train=False, download=True, transform=transform)
# 加载数据集并设置批次大小
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)
# 修正 print 语句，明确输出内容
print(
    f"训练集的 batch 数量: {len(train_loader)}，每个 batch 中的数据数量: {train_loader.batch_size}"
)
print(
    f"测试集的 batch 数量: {len(test_loader)}，每个 batch 中的数据数量: {test_loader.batch_size}"
)

Files already downloaded and verified
Files already downloaded and verified
训练集的 batch 数量: 782，每个 batch 中的数据数量: 64
测试集的 batch 数量: 157，每个 batch 中的数据数量: 64


In [ ]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import Flatten
from torch.nn import Dropout
from torch.nn import Sequential
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

class VggNet(Module):
    def vgg_block(self, num_convs, in_channels, out_channels):
        layers = []
        for _ in range(num_convs):
            layers.append(Conv2d(in_channels, out_channels, kernel_size=(3, 3), padding=(1, 1)))
            layers.append(ReLU())
            in_channels = out_channels
        layers.append(MaxPool2d(kernel_size=(2, 2), stride=(2, 2)))
        return Sequential(*layers)
        
    def __init__(self):
        super().__init__()
        # VGGNet11的配置
        # conv_arch = [(1, 64), (1, 128), (2, 256), (2, 512), (2, 512)]
        # VGGNet13的配置
        # conv_arch = [(2, 64), (2, 128), (2, 256), (2, 512), (2, 512)]
        # VGGNet16的配置
        # conv_arch = [(2, 64), (2, 128), (3, 256), (3, 512), (3, 512)]
        # VGGNet19的配置
        conv_arch = [(2, 64), (2, 128), (4, 256), (4, 512), (4, 512)]
        in_channels = 3
        # 定义特征提取部分
        self.features = Sequential()
        for i, (num_convs, out_channels) in enumerate(conv_arch):
            conv_blk = self.vgg_block(num_convs, in_channels, out_channels)
            self.features.add_module(f'conv_block_{i}', conv_blk)
            in_channels = out_channels
        # 对特征提取部分的卷积层进行 Kaiming 初始化
        for layer in self.features:
            if isinstance(layer, Conv2d):
                kaiming_uniform_(layer.weight, nonlinearity='relu')
        # 定义分类器部分
        self.classifier = Sequential(
            Flatten(),
            Linear(512 * 7 * 7, 4096),
            ReLU(),
            Dropout(p=0.5),
            Linear(4096, 4096),
            ReLU(),
            Dropout(p=0.5),
            Linear(4096, 10)  # 假设是 10 分类任务
        )
        # 对分类器部分的全连接层进行 Xavier 初始化
        for layer in self.classifier:
            if isinstance(layer, Linear):
                if layer is not self.classifier[-1]:
                    kaiming_uniform_(layer.weight, nonlinearity='relu')
                else:
                    xavier_uniform_(layer.weight)
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [3]:
# 训练模型
# 定义损失函数和优化器
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
import time


def train_model(train_loader, model):
    # 定义损失函数和优化器
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # 训练的轮数
    num_epochs = 10
    # 初始化 running_loss
    running_loss = 0.0
    # 记录训练开始时间
    start_time = time.time()
    # 遍历epoch
    for epoch in range(num_epochs):
        # 遍历训练数据
        for i, (inputs, labels) in enumerate(train_loader):
            # 将数据移动到GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            # 梯度清零
            optimizer.zero_grad()
            # 计算模型输出
            y_hat = model(inputs)
            # 计算损失
            loss = criterion(y_hat, labels)
            # 反向传播
            loss.backward()
            # 更新参数
            optimizer.step()
            running_loss += loss.item()
        # 打印每个 epoch 的损失
        print(
            f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')
        # 每个 epoch 结束后重置 running_loss
        running_loss = 0.0
    # 记录训练结束时间
    end_time = time.time()
    # 计算训练总时间
    total_time = end_time - start_time
    # 转换为分钟和秒
    minutes = int(total_time // 60)
    seconds = total_time % 60
    print(
        f'Training completed in {minutes} minutes and {seconds:.2f} seconds.')

In [4]:
# 评估模型
from sklearn.metrics import accuracy_score
from numpy import vstack
from numpy import argmax


def evaluate_model(test_loader, model):
    predictions, actuals = list(), list()
    for i, (inputs, labels) in enumerate(test_loader):
        # 将数据移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 计算模型输出
        y_hat = model(inputs)
        # 转换为 numpy 数据类型
        y_hat = y_hat.detach().cpu().numpy()
        actual = labels.cpu().numpy()
        # 转换为类标签
        y_hat = argmax(y_hat, axis=1)
        # 为stack格式化
        actual = actual.reshape((len(actual), 1))
        y_hat = y_hat.reshape((len(y_hat), 1))
        # 存储
        predictions.append(y_hat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # 计算准确率
    acc = accuracy_score(actuals, predictions)
    return acc

In [5]:
# 产生实例,并且将实例放入GPU
VggNet_model = VggNet().to(device)
# 训练实例
train_model(train_loader, VggNet_model)
# 评估实例整体准确率
acc = evaluate_model(test_loader, VggNet_model)
print('Overall Accuracy: %.3f' % acc)

Epoch 1/10, Loss: 2.3040281296386134
Epoch 2/10, Loss: 2.303758852317205
Epoch 3/10, Loss: 2.3036516770682374
Epoch 4/10, Loss: 2.303596309078929
Epoch 5/10, Loss: 2.3034174064236224
Epoch 6/10, Loss: 2.3033584716070035
Epoch 7/10, Loss: 2.303367879384619
Epoch 8/10, Loss: 2.3034772220474986
Epoch 9/10, Loss: 2.3036128797799424
Epoch 10/10, Loss: 2.303322779248133
Training completed in 41 minutes and 23.63 seconds.
Overall Accuracy: 0.101
